In [1]:

import numpy as np
ZERO_KEY = "<ZERO>"

def hotvec_dict(word_dict):
    word_dict = dict(filter(lambda elem: elem[1] > THRESHOLD, word_dict.items()))
    vec_mat = np.eye(len(word_dict))
    return {k: vec_mat[list(word_dict.keys()).index(k)] for k in word_dict}, word_dict

hotvec, word_dict = hotvec_dict(word_dict)
hotvec.update({ZERO_KEY:np.zeros([1,len(word_dict)],dtype=float)})

In [5]:
## X train
from nltk.tokenize import word_tokenize
import string

THRESHOLD_TXT_WORD = 10
BASE_PATH = "42bin_haber/news"
TARGET_PATH = "cleared_samples"
SAMPLE_FILE_NAME = "all_words.txt"

PATHS_OF_CLASSES = [os.path.join(BASE_PATH, file) for file in os.listdir(BASE_PATH)]

def get_training_data(hotvec, PATHS_OF_CLASSES, word_dict):
    
    vec_mat_y = np.eye(len(PATHS_OF_CLASSES))
    dict_y = {basename(path): vec_mat_y[PATHS_OF_CLASSES.index(path)] for path in PATHS_OF_CLASSES}
    
    training_data = []
    ##
    folder = 0
    ##
    for path in PATHS_OF_CLASSES:
        class_name = basename(path)
        y_data = dict_y[class_name]
        
        ##
        fil = 0
        ##
        for file in os.listdir(path):
            fil+=1
            print(class_name,":",fil)
            if file.endswith('.txt'):
                with open(os.path.join(path, file),'r',encoding="utf-8",errors='ignore') as f:
                    text = f.read()
                    word_list = clear_sample(text, word_dict)   ## you can change clear type by writing new function
                    X_data = [hotvec[w] for w in word_list]
                    training_data.append([X_data, y_data])
    
    return training_data
                    
        
def clear_sample(text, word_dict):
    remov = set(stopwords.words("turkish"))
    words = word_tokenize(text)
    result_words = []
    for w in words:
        temp = w.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))).split()
        result_words.extend(t for t in temp)
        
    words = [word.lower() for word in result_words]
    words = [w.strip() for w in words if '\n' or '' != w]
    words = [w for w in words if w not in remov]
    words = [x for x in words if not any(c.isdigit() for c in x)]
    words = filter (lambda word: len (word) > 2, words)
    
    w_list = []
    
    for w in words:
        if w in word_dict:
            w_list.append(w)
        if len(w_list) == THRESHOLD_TXT_WORD:
            break
    
    while len(w_list) < THRESHOLD_TXT_WORD:
        w_list.append(ZERO_KEY)
        
    return w_list


In [ ]:
training_data = get_training_data(hotvec, PATHS_OF_CLASSES, word_dict)

In [8]:
np.random.shuffle(training_data)
np.save("training_data_shuffled.npy", training_data)

D:\Anaconda\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
